## 1 Importar liberias

In [261]:
import bs4 as bs 
import requests
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
import html5lib as html5lib

## 2 Carga de Tablas 

FORMA 1

In [93]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks')
soup = bs.BeautifulSoup(response.text, "lxml")
Tabla = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in Tabla.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

In [ ]:
tickers = [s.replace('\n', '') for s in tickers]
start = datetime.datetime(2000,1,1)
end = datetime.datetime(2021,12,31)
df_list = list()
for ticker in tickers:
    data = yf.download(ticker, group_by="Ticker", start=start, end=end)
    data['ticker'] = ticker  # add this column because the dataframe doesn't contain a column with the ticker
    df_list.append(data)
    sp500 = pd.concat(df_list)

In [314]:
sp500.to_csv('sp500.csv', decimal=',',sep=';')


In [315]:
SP500 = pd.read_csv('sp500.csv', decimal=',',sep=';')

FORMA 2

In [263]:
SPWiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks')
SPWiki = SPWiki[0]

In [184]:
SPWiki.to_csv('SPWiki.csv')

In [264]:
SPWiki = pd.read_csv('SPWiki.csv')

In [265]:
SPWiki.head(5)

,Unnamed: 0,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


## 3 Orden de Tablas

In [301]:
SPWiki.rename(columns={'Symbol':'ticker'}, inplace=True)

In [ ]:
SPWiki.drop(columns=['Unnamed: 0', 'Security', 'SEC filings','Headquarters Location', 'Date first added', 'CIK', 'Founded'], inplace=True)

In [316]:
SP500.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,ticker
0,2000-01-03,48.03125,48.25000,47.03125,47.1875,26.434845,2173400.0,MMM
1,2000-01-04,46.43750,47.40625,45.31250,45.3125,25.384466,2713800.0,MMM
2,2000-01-05,45.56250,48.12500,45.56250,46.6250,26.119745,3699400.0,MMM
3,2000-01-06,47.15625,51.25000,47.15625,50.3750,28.220518,5975800.0,MMM
4,2000-01-07,50.56250,51.90625,49.96875,51.3750,28.780733,4101200.0,MMM


In [317]:
SP500 = SP500[['ticker','Date','Open','High','Low','Close','Adj Close', 'Volume']]

In [318]:
SP500.head(5)

,ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,MMM,2000-01-03,48.03125,48.25000,47.03125,47.1875,26.434845,2173400.0
1,MMM,2000-01-04,46.43750,47.40625,45.31250,45.3125,25.384466,2713800.0
2,MMM,2000-01-05,45.56250,48.12500,45.56250,46.6250,26.119745,3699400.0
3,MMM,2000-01-06,47.15625,51.25000,47.15625,50.3750,28.220518,5975800.0
4,MMM,2000-01-07,50.56250,51.90625,49.96875,51.3750,28.780733,4101200.0


In [319]:
fecha = pd.to_datetime(SP500['Date'])
SP500['Dia'] = fecha.dt.day_name()
SP500['Mes'] = fecha.dt.month
SP500['Año'] = fecha.dt.year

## 4 Calculos

1 Cual es el mejor día para invertir teniendo en cuenta el retorno de los movimiento gap?

In [320]:
SP500['Gaps'] = round((np.log(SP500['Open']/SP500['Close'].shift(1)).fillna(0)),4)
SP500['Intradiarios'] = round((np.log(SP500['Close']/SP500['Open']).fillna(0)),4)
SP500['Variaciones'] = round((SP500['Adj Close'].pct_change()),4)
SP500['Volatilidad'] = round((SP500.Variaciones.rolling(250,min_periods=1).std()*100*(250)**0.5),4)

In [321]:
SP500 = SP500[['ticker','Date','Año','Mes','Dia','Open','High','Low','Close','Adj Close', 'Volume','Gaps','Variaciones','Volatilidad','Intradiarios']]

In [322]:
SP500.fillna(0, inplace=True)

In [323]:
Lunes = SP500['Dia'] == 'Monday'
dflunes = SP500[Lunes]
dflunes.head(5)

,ticker,Date,Año,Mes,Dia,Open,High,Low,Close,Adj Close,Volume,Gaps,Variaciones,Volatilidad,Intradiarios
0,MMM,2000-01-03,2000,1,Monday,48.03125,48.25000,47.03125,47.1875,26.434845,2173400.0,0.0000,0.0000,0.0000,-0.0177
5,MMM,2000-01-10,2000,1,Monday,50.21875,51.75000,50.00000,51.1250,28.640684,3863800.0,-0.0228,-0.0049,70.1180,0.0179
14,MMM,2000-01-24,2000,1,Monday,47.90625,47.90625,45.81250,46.3750,25.979691,2971000.0,0.0092,-0.0230,49.0421,-0.0325
19,MMM,2000-01-31,2000,1,Monday,46.12500,48.68750,46.03125,46.8125,26.224781,3282800.0,0.0000,0.0149,49.4461,0.0148
24,MMM,2000-02-07,2000,2,Monday,44.62500,44.68750,43.31250,43.8125,24.544149,3935400.0,0.0021,-0.0161,45.3381,-0.0184


In [328]:
dflunes['Gaps'].sum()

-729.2625999999997

In [330]:
Martes = SP500['Dia'] == 'Tuesday'
dfmartes = SP500[Martes]

Miercoles = SP500['Dia'] == 'Wednesday'
dfmiercoles = SP500[Miercoles]

Jueves = SP500['Dia'] == 'Thursday'
dfjueves = SP500[Jueves]

Viernes = SP500['Dia'] == 'Friday'
dfviernes = SP500[Viernes]

print(dfmartes['Gaps'].sum(),
dfmiercoles['Gaps'].sum(),
dfjueves['Gaps'].sum(),
dfviernes['Gaps'].sum())

264.8635 17.793 -49.3723 49.882


RESPUESTA: Lunes.

2 Cual es el mejor día para invertir teniendo en cuenta el retorno de los movimientos intradiarios?

In [278]:
print(dflunes['Intradiarios'].sum(), dfmartes['Intradiarios'].sum(),
dfmiercoles['Intradiarios'].sum(),
dfjueves['Intradiarios'].sum(),
dfviernes['Intradiarios'].sum())

-143.3051289329919 93.9169935520954 122.42030293215832 259.1565318600915 123.55967147611804


RESPUESTA: Jueves.

3 Cuales son las mejores industrias que pertenecen al SP500 en las cuales se puede invertir?

In [332]:
dfSector = SP500.merge(SPWiki, how='inner', on='ticker')
dfpbi = dfSector['GICS Sector'].to_csv('dfpbi.csv')

In [333]:
ind = dfSector['GICS Sector'] == 'Industrials'
dfInd = dfSector[ind] 
a2021 = dfInd['Año'] == 2021
dfInd2021 = dfInd[a2021]
dfInd2021.head(5)


,ticker,Date,Año,Mes,Dia,Open,High,Low,Close,Adj Close,Volume,Gaps,Variaciones,Volatilidad,Intradiarios,GICS Sector,GICS Sub-Industry
5284,MMM,2021-01-04,2021,1,Monday,175.000000,176.199997,170.550003,171.869995,163.336884,2996200.0,0.0012,-0.0167,37.4236,-0.0180,Industrials,Industrial Conglomerates
5285,MMM,2021-01-05,2021,1,Tuesday,172.009995,173.250000,170.649994,171.580002,163.061264,2295300.0,0.0008,-0.0017,37.3937,-0.0025,Industrials,Industrial Conglomerates
5286,MMM,2021-01-06,2021,1,Wednesday,172.720001,175.570007,172.039993,174.190002,165.541718,3346400.0,0.0066,0.0152,37.4225,0.0085,Industrials,Industrial Conglomerates
5287,MMM,2021-01-07,2021,1,Thursday,171.559998,173.460007,166.160004,169.720001,161.293625,5863400.0,-0.0152,-0.0257,37.5102,-0.0108,Industrials,Industrial Conglomerates
5288,MMM,2021-01-08,2021,1,Friday,169.169998,169.539993,164.610001,166.619995,158.347534,4808100.0,-0.0032,-0.0183,37.5550,-0.0152,Industrials,Industrial Conglomerates


In [334]:
dfInd2021.sort_values('Variaciones', inplace=True, ascending=False)
dfInd2021.head(10)

C:\Users\kmyde\AppData\Local\Temp\ipykernel_19924\3932425615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfInd2021.sort_values('Variaciones', inplace=True, ascending=False)


,ticker,Date,Año,Mes,Dia,Open,High,Low,Close,Adj Close,Volume,Gaps,Variaciones,Volatilidad,Intradiarios,GICS Sector,GICS Sub-Industry
872899,EFX,2021-04-22,2021,4,Thursday,217.899994,226.929993,212.690002,221.410004,219.588364,3209100.0,0.1233,0.1494,33.1533,0.0160,Industrials,Research & Consulting Services
1037672,GNRC,2021-02-11,2021,2,Thursday,293.500000,318.750000,293.000000,316.109985,316.109985,1619800.0,0.0559,0.1390,55.2778,0.0742,Industrials,Electrical Components & Equipment
1897752,PWR,2021-09-02,2021,9,Thursday,110.269997,114.730003,108.720001,114.040001,113.851120,3668800.0,0.0802,0.1206,32.5583,0.0336,Industrials,Construction & Engineering
11026,AOS,2021-10-28,2021,10,Thursday,67.029999,72.720001,67.029999,70.690002,70.115486,2735400.0,0.0424,0.1050,27.8352,0.0532,Industrials,Building Products
1744871,PCAR,2021-01-21,2021,1,Thursday,92.919998,103.190002,91.879997,98.739998,94.902901,8088100.0,0.0387,0.1046,37.9035,0.0608,Industrials,Construction Machinery & Heavy Trucks
2267557,UPS,2021-04-27,2021,4,Tuesday,190.259995,197.279999,189.710007,194.130005,188.249084,14666500.0,0.0790,0.1042,32.5164,0.0201,Industrials,Air Freight & Logistics
703686,DE,2021-02-19,2021,2,Friday,320.000000,335.209991,315.869995,330.000000,324.357880,4870100.0,0.0637,0.0991,46.9835,0.0308,Industrials,Agricultural & Farm Machinery
156595,AAL,2021-02-22,2021,2,Monday,19.840000,21.110001,19.309999,20.440001,20.440001,99937700.0,0.0602,0.0942,104.5738,0.0298,Industrials,Airlines
156579,AAL,2021-01-28,2021,1,Thursday,20.330000,21.770000,16.820000,18.100000,18.100000,319607600.0,0.2051,0.0930,104.6503,-0.1162,Industrials,Airlines
707578,DAL,2021-12-02,2021,12,Thursday,34.410000,36.669998,33.759998,36.639999,36.639999,36044800.0,0.0259,0.0928,36.5724,0.0628,Industrials,Airlines


In [282]:
EFX = dfInd2021['ticker'] == 'EFX'  
dfEFX = dfInd2021[EFX]

GNRC = dfInd2021['ticker'] == 'GNRC'  
dfGNRC = dfInd2021[GNRC]

PWR = dfInd2021['ticker'] == 'PWR'  
dfPWR = dfInd2021[PWR]

AOS = dfInd2021['ticker'] == 'AOS'  
dfAOS = dfInd2021[AOS]

PCAR = dfInd2021['ticker'] == 'PCAR'  
dfPCAR = dfInd2021[PCAR]

print(dfEFX['Variaciones'].sum(),
      dfGNRC['Variaciones'].sum(),
      dfPWR['Variaciones'].sum(),
      dfAOS['Variaciones'].sum(),
      dfPCAR['Variaciones'].sum())

0.46146708059699804 0.5345671914729615 0.502311001073648 0.4976139123934378 0.07524557480478666


In [283]:
print(dfEFX['Volatilidad'].sum(),
      dfGNRC['Volatilidad'].sum(),
      dfPWR['Volatilidad'].sum(),
      dfAOS['Volatilidad'].sum(),
      dfPCAR['Volatilidad'].sum())

8232.126807398534 11725.714218923065 9067.627846011621 7724.95870031873 7016.086085765966


In [284]:
print(dfEFX['Gaps'].sum(),
      dfGNRC['Gaps'].sum(),
      dfPWR['Gaps'].sum(),
      dfAOS['Gaps'].sum(),
      dfPCAR['Gaps'].sum())

0.24547785145539583 0.5597118535534502 0.5151704510460706 0.3430831075996561 0.28004997436791357


RESPUESTA: se encuentra en power bi

4 Cuales fueron los momentos de alta volatilidad que afectaron al SP500?

In [325]:
SP500.tail(5)

,ticker,Date,Año,Mes,Dia,Open,High,Low,Close,Adj Close,Volume,Gaps,Variaciones,Volatilidad,Intradiarios
2467087,ZTS,2021-12-23,2021,12,Thursday,241.699997,244.389999,240.690002,242.509995,241.697968,1172400.0,0.0005,0.0039,19.7495,0.0033
2467088,ZTS,2021-12-27,2021,12,Monday,244.389999,246.720001,243.080002,246.509995,245.684570,807000.0,0.0077,0.0165,19.8029,0.0086
2467089,ZTS,2021-12-28,2021,12,Tuesday,247.000000,247.839996,243.639999,244.250000,243.432159,1004400.0,0.0020,-0.0092,19.8258,-0.0112
2467090,ZTS,2021-12-29,2021,12,Wednesday,244.020004,247.850006,243.639999,247.029999,246.202835,939900.0,-0.0009,0.0114,19.8411,0.0123
2467091,ZTS,2021-12-30,2021,12,Thursday,247.050003,249.270004,244.960007,245.160004,244.339111,859100.0,0.0001,-0.0076,19.8188,-0.0077


In [327]:
dfvol = SP500.to_csv('dfvol.csv',sep=';', decimal=',')

RESPUESTA: se encuentra en power bi

5 Cuales son las 9 mejores empresas para invertir?

In [173]:
a2021 = SP500['Año'] == 2021
df2021 = SP500[a2021]
df2021.sort_values('Variaciones', inplace=True, ascending=False)
df2021.head(10)

C:\Users\kmyde\AppData\Local\Temp\ipykernel_19924\135226154.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2021.sort_values('Variaciones', inplace=True, ascending=False)


,ticker,Date,Año,Mes,Dia,Open,High,Low,Close,Adj Close,Volume,Gaps,Variaciones,Volatilidad,Intradiarios
350299,BIIB,2021-06-07,2021,6,Monday,295.350006,468.549988,286.000000,395.850006,395.850006,17954100.0,0.031680,0.383414,71.707009,0.292874
1417433,LUMN,2021-01-26,2021,1,Tuesday,11.600000,15.380000,11.370000,14.590000,12.939567,60245800.0,0.018269,0.280948,59.553870,0.229331
853934,ENPH,2021-10-27,2021,10,Wednesday,205.500000,225.399994,204.039993,216.220001,216.220001,15168400.0,0.169499,0.246512,69.388018,0.050850
1539773,MRNA,2021-11-26,2021,11,Friday,319.339996,350.149994,313.000000,329.630005,329.630005,35901200.0,0.155357,0.205713,79.806085,0.031714
246265,ANET,2021-11-02,2021,11,Tuesday,122.394997,131.250000,120.760002,122.967499,122.967499,13023600.0,0.180885,0.203882,32.067845,0.004667
2170894,TSLA,2021-03-09,2021,3,Tuesday,608.179993,678.090027,595.210022,673.580017,673.580017,67523300.0,0.077191,0.196412,84.857225,0.102136
1095583,HIG,2021-03-18,2021,3,Thursday,57.939999,68.949997,57.700001,68.150002,66.336708,23914200.0,0.009189,0.187075,50.587667,0.162303
867524,EPAM,2021-12-08,2021,12,Wednesday,648.869995,704.719971,643.330017,700.140015,700.140015,3746100.0,0.086671,0.176706,35.436832,0.076048
1539696,MRNA,2021-08-09,2021,8,Monday,411.300995,493.756989,409.660004,484.470001,484.470001,42269900.0,-0.005864,0.171009,76.251393,0.163730
1611378,NFLX,2021-01-20,2021,1,Wednesday,565.419983,593.289978,556.859985,586.340027,586.340027,32637500.0,0.119427,0.168543,48.684184,0.036331


In [45]:
#4
#segunda mitad del 2000

In [331]:
a2021 = SP500['Año'] == 2021
df2021 = SP500[a2021]
df2021.sort_values('Variaciones', inplace=True, ascending=False)
df2021.head(10)

C:\Users\kmyde\AppData\Local\Temp\ipykernel_19924\135226154.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2021.sort_values('Variaciones', inplace=True, ascending=False)


,ticker,Date,Año,Mes,Dia,Open,High,Low,Close,Adj Close,Volume,Gaps,Variaciones,Volatilidad,Intradiarios
350299,BIIB,2021-06-07,2021,6,Monday,295.350006,468.549988,286.000000,395.850006,395.850006,17954100.0,0.0317,0.3834,71.7053,0.2929
1417433,LUMN,2021-01-26,2021,1,Tuesday,11.600000,15.380000,11.370000,14.590000,12.939567,60245800.0,0.0183,0.2809,59.5498,0.2293
853934,ENPH,2021-10-27,2021,10,Wednesday,205.500000,225.399994,204.039993,216.220001,216.220001,15168400.0,0.1695,0.2465,69.3836,0.0509
1539773,MRNA,2021-11-26,2021,11,Friday,319.339996,350.149994,313.000000,329.630005,329.630005,35901200.0,0.1554,0.2057,79.8049,0.0317
246265,ANET,2021-11-02,2021,11,Tuesday,122.394997,131.250000,120.760002,122.967499,122.967499,13023600.0,0.1809,0.2039,32.0678,0.0047
2170894,TSLA,2021-03-09,2021,3,Tuesday,608.179993,678.090027,595.210022,673.580017,673.580017,67523300.0,0.0772,0.1964,84.8519,0.1021
1095583,HIG,2021-03-18,2021,3,Thursday,57.939999,68.949997,57.700001,68.150002,66.336708,23914200.0,0.0092,0.1871,50.5851,0.1623
867524,EPAM,2021-12-08,2021,12,Wednesday,648.869995,704.719971,643.330017,700.140015,700.140015,3746100.0,0.0867,0.1767,35.4331,0.0760
1539696,MRNA,2021-08-09,2021,8,Monday,411.300995,493.756989,409.660004,484.470001,484.470001,42269900.0,-0.0059,0.1710,76.2500,0.1637
1611378,NFLX,2021-01-20,2021,1,Wednesday,565.419983,593.289978,556.859985,586.340027,586.340027,32637500.0,0.1194,0.1685,48.6816,0.0363


In [47]:
BIIB = df2021['ticker'] == 'BIIB'  
dfBIIB = df2021[BIIB]
dfBIIB.head(5)

,ticker,Date,Año,Mes,Dia,Open,High,Low,Close,Adj Close,Volume,Gaps,Variaciones,Volatilidad,Intradiarios
352784,BIIB,2021-06-07,2021,6,Monday,295.350006,468.549988,286.000000,395.850006,395.850006,17954100.0,0.031680,0.383414,71.707009,0.292874
352927,BIIB,2021-12-29,2021,12,Wednesday,235.000000,265.540009,234.130005,258.309998,258.309998,6869700.0,-0.004204,0.094580,48.595014,0.094575
352683,BIIB,2021-01-11,2021,1,Monday,255.860001,275.489990,253.309998,267.519989,267.519989,2999200.0,0.013061,0.059317,69.488007,0.044564
352696,BIIB,2021-01-29,2021,1,Friday,294.429993,296.380005,281.510010,282.609985,282.609985,4789400.0,0.094539,0.055027,69.778156,-0.040973
352686,BIIB,2021-01-14,2021,1,Thursday,269.489990,280.489990,268.000000,280.290009,280.290009,1520400.0,0.010519,0.051074,69.599215,0.039294


In [48]:
dfBIIB['Variaciones'].sum()

0.08556580090190957

In [49]:
LUMN = df2021['ticker'] == 'LUMN'  
dfLUMN = df2021[LUMN]

ENPH = df2021['ticker'] == 'ENPH'  
dfENPH = df2021[ENPH]

MRNA = df2021['ticker'] == 'MRNA'  
dfMRNA = df2021[MRNA]

ANET = df2021['ticker'] == 'ANET'  
dfANET = df2021[ANET]

TSLA = df2021['ticker'] == 'TSLA'  
dfTSLA = df2021[TSLA]

HIG = df2021['ticker'] == 'HIG'  
dfHIG = df2021[HIG]

EPAM = df2021['ticker'] == 'EPAM'  
dfEPAM = df2021[EPAM]

NFLX = df2021['ticker'] == 'NFLX'  
dfNFLX = df2021[NFLX]

In [50]:
print(dfBIIB['Variaciones'].sum(),
      dfLUMN['Variaciones'].sum(),
      dfENPH['Variaciones'].sum(),
      dfMRNA['Variaciones'].sum(),
      dfANET['Variaciones'].sum(),
      dfTSLA['Variaciones'].sum(),
      dfHIG['Variaciones'].sum(),
      dfEPAM['Variaciones'].sum(),
      dfNFLX['Variaciones'].sum())

0.08556580090190957 0.4714987767209089 0.2838056449191555 1.202367581685544 0.73422085864348 0.5641525568000483 0.4106873084668361 0.6944925206384478 0.1738570569489175


RESPUESTA: Se encuentra en power bi